In [27]:
from vision_text_dual_encoder import VisionTextDualEncoderConfig, VisionTextDualEncoderModel
from transformers import (
    AutoImageProcessor,
    AutoModel,
    AutoTokenizer,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
)
import torch

In [28]:
model_path = 'blip2_t5_finetuned'

In [30]:
model = VisionTextDualEncoderModel.from_pretrained(
    model_path,
    config=VisionTextDualEncoderConfig.from_pretrained(model_path),
    torch_dtype=torch.bfloat16,
    device_map='cuda'
).eval()

tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    use_fast=False,
)

image_processor = AutoImageProcessor.from_pretrained(
    model_path
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [67]:
from PIL import Image
import requests

img_url = 'https://sun1-92.userapi.com/impg/fnn1LcU9RbFGAk8Y64_WCB0nrAzjJKFa-5EpZg/STS6yLrU-XI.jpg?size=640x480&quality=95&sign=dab78f730e79022d36a499e0ae1783be&type=album'
image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

text_inputs = tokenizer(["A man is walking with a dog", "A man write in notebook"], return_tensors="pt", padding=True)
image_inputs = image_processor(image, return_tensors="pt")

In [68]:
with torch.no_grad():
    out = model(**image_inputs.to('cuda'), **text_inputs.to('cuda'))

In [69]:
torch.nn.functional.softmax(out.logits_per_image)

/tmp/ipykernel_3351/767894517.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  torch.nn.functional.softmax(out.logits_per_image)


tensor([[0.5234, 0.4766]], device='cuda:0', dtype=torch.bfloat16)

In [63]:
out.text_embeds @ out.image_embeds.T

tensor([[0.1621],
        [0.2207]], device='cuda:0', dtype=torch.bfloat16)